# PROVIDE A RANDOMLY DISTRIBUTED DELAY TO ACTIVITIES

In [1]:
# Import dependencies
import datetime
import dateutil
from openclsim.appendix import get_event_log
import openclsim.model as model
import openclsim.plugins as plugins
import pandas as pd
import scipy.stats as st
import simpy

In [2]:
# Create a simulation environment
class SimulationEnvironment(object):
    """
    A OpenCLSim simulation object.

    Use the class methods to define the activities and start the
    simulation. In this example, a basic activity is modelled using 
    the `model.BasicActivity` class and the length of the activity is
    extended using a random delay with the `model.HasDelayPlugin`. For
    further reference, check the `DelayBasicActivity` class below.

    Parameters
    ----------
        start_date: datetime.datetime
            A datetime object specifying the simulation start date.
    """
    def __init__(self, start_date: datetime.datetime):

        # Define start date and time of simulation
        self.start_date = start_date
        start_utc = start_date.replace(tzinfo=dateutil.tz.UTC)
        start_epoch = start_utc.timestamp()

        # Initialise a SimPy simulation environment
        self.env = simpy.Environment(initial_time=start_epoch)
        self.registry = {}

    def define_activities(self):
        """Use this function to define your activities."""
        # In this example we define a basic activity
        length = 60  # The activity takes about 5 hours.
        delay_by = st.uniform(loc=0, scale=1)  # The delay as RANDOM variable.

        activity = DelayedBasicActivity(
            env=self.env,   # The SimPy environment
            registry=self.registry,  
            name="A basic activity",  # Description
            duration=length,  # The length the activity takes
            delay_percentage=delay_by  # The delay in percentages
        )

        # The while activity allows us to repeat a certain list of activities
        while_activity = model.WhileActivity(
            env=self.env,  # The SimPy environment
            registry=self.registry,
            name="while",  # Description, must match `while` else won't work.
            sub_processes=[activity],  # The activities involved in loop
            condition_event=[{"type": "activity", "name": "while", "state": "done"}]
        )

        # Register the activities
        model.register_processes([while_activity]) 
        
        # Return the base activity for logging purposes
        return [activity]  
        
    def execute_simulation(self):
        """Function starts the simulation."""
        self.activities = self.define_activities()
        self.env.run(until=self.env.now + 60 * 60)
        return 'SUCCESSFUL'

    @property
    def event_log(self):
        """Function returns the event log."""
        return get_event_log(activity_list=self.activities)

    @property
    def project_length(self):
        """Function returns the project length."""
        log = self.event_log
        dt = (log.Timestamp.iloc[-1] - log.Timestamp.iloc[0])
        return dt.total_seconds() / (3600)


# Define the delayed basic activity using inheritance.
class DelayedBasicActivity(plugins.HasDelayPlugin, model.BasicActivity):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [3]:
sim = SimulationEnvironment(datetime.datetime(2021, 1, 1))
res = sim.execute_simulation()
log = sim.event_log
log

,Timestamp,ActivityState,Description,ActivityID
0,2021-01-01 00:00:00.000000,START,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
1,2021-01-01 00:01:00.000000,STOP,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
2,2021-01-01 00:01:00.000000,WAIT_START,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
3,2021-01-01 00:01:00.174983,WAIT_STOP,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
4,2021-01-01 00:01:00.174983,START,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
...,...,...,...,...
232,2021-01-01 00:58:17.871131,START,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
233,2021-01-01 00:59:17.871131,STOP,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
234,2021-01-01 00:59:17.871131,WAIT_START,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a
235,2021-01-01 00:59:18.442386,WAIT_STOP,A basic activity,9300c98b-8a2a-4a68-b4cf-f9307013316a


## PROOF THAT DELAY LENGTH IS ACTUALLY VARYING

In [4]:
import plotly.graph_objects as go

In [5]:
periods = []

for i in range(1000):
    sim = SimulationEnvironment(datetime.datetime(2021, 1, 1))
    res = sim.execute_simulation()
    periods.append(sim.project_length)

In [6]:
hist = go.Histogram(
    x=periods, 
    autobinx=True, 
    marker=dict(
        color='red',
        opacity=0.75,
        line=dict(color='black', width=1)
    ),
    cumulative=dict(enabled=True),
    histnorm='probability'
)

fig = go.Figure(data=hist)

fig.update_layout(
    xaxis_title = 'Estimated Project Length [hrs]',
    yaxis_title = 'Probability [-]',
    title='Project Length Distribution for 1000 simulations'
)

fig.show()

<div style="width:500px;">

Note how the above image transforms to a normal distribution, whilst the delay was described using an uniform distribution. This is due to the central limit theorem, which establishes that, in many situations, when independent random variables are added their sum tends toward a normal distribution. If the `sub_processes` in the `model.SequentialActivity` contains only a single activity, we would see the uniform distribution!

</div>